In [36]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
import random
from bs4 import BeautifulSoup
import sqlite3
import inspect
import re
import time
from datetime import datetime

driver = webdriver.Chrome()
driver.get("https://x.com/")

for_unfollow = []

In [17]:
def days_to_now(specific_date_str):
    specific_date = datetime.strptime(specific_date_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    current_date = datetime.now()
    difference = current_date - specific_date
    days_difference = difference.days
    return days_difference

In [18]:
def record_count():
    with sqlite3.connect('following.db') as connection:
        cursor = connection.cursor()
        try:
            cursor.execute("SELECT COUNT(*) FROM following_table")
            count = cursor.fetchone()[0]
        finally:
            cursor.close()
    return count

In [19]:
def get_record_at_position(n):
    with sqlite3.connect('following.db') as connection:
        cursor = connection.cursor()
        try:
            query = "SELECT user_name FROM following_table LIMIT 1 OFFSET ?"
            cursor.execute(query, (n - 1,))
            record = cursor.fetchone()
            if record:
                {}
            else:
                return False
        finally:
            cursor.close()
    return record[0]

In [20]:
def delete_record_at_position(n):
    with sqlite3.connect('following.db') as connection:
        cursor = connection.cursor()
        try:
            fetch_query = "SELECT rowid FROM following_table LIMIT 1 OFFSET ?"
            cursor.execute(fetch_query, (n - 1,))
            record = cursor.fetchone()
            if record:
                delete_query = "DELETE FROM following_table WHERE rowid = ?"
                cursor.execute(delete_query, (record[0],))
                connection.commit()
            else:
                return False
        finally:
            cursor.close()
    return True

In [21]:
def is_inactive(user_link, treshold = 150):
    driver.get(user_link + '/with_replies')
    time.sleep(10)
    elements = driver.find_elements(By.TAG_NAME, 'time')
    tweet_times = set()
    for element in elements:
        try:
            datetime_value = element.get_attribute('datetime')
            tweet_times.add(days_to_now(datetime_value))
        except:
            pass
    latest_tweet = min(tweet_times)
    if (latest_tweet > treshold):
        return True
    else:
        return False

In [27]:
def do_unfollow():
    try:
        wait = WebDriverWait(driver, random.randint(10, 20))
        unfollow_button = wait.until(
            EC.presence_of_element_located((By.XPATH, "//button[contains(., 'Following')]"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", unfollow_button)
        time.sleep(5)  # Optional: Add a small delay to ensure the element is in view
        # Click the button using JavaScript
        driver.execute_script("arguments[0].click();", unfollow_button)
        time.sleep(3)
        confirm_unfollow = driver.find_element(By.CSS_SELECTOR, "button.css-175oi2r[data-testid='confirmationSheetConfirm'][role='button']")
        confirm_unfollow.click()
        time.sleep(random.randint(120, 240))
        return True
    except:
        return False

In [23]:
def sleep_time(i):
    if i % 10 == 0:
        return (random.randint(600, 1200))  # 15 minutes
    elif i % 5 == 0:
        return (random.randint(150, 480))  # 5 minutes
    else:
        return (random.randint(60, 120))  # 1 minute

In [37]:
def do_unfollow_all_inactive_followings():
    num_deleted_items = 0
    itration = 0
    while itration < record_count():
        itration += 1
        user_link = get_record_at_position(itration)
        user_name = user_link.split('/')[-1]
        try:
            if(is_inactive(user_link)):
                do_unfollow()
                num_deleted_items += 1
                print(num_deleted_items, ' - ', user_name,  '  following was deleted! *************')
            else:
                print(itration, ' - ', user_name, '  is active.')
        except:
            pass
        time.sleep(sleep_time(itration))
    print('num_deleted_items  =  ', num_deleted_items)
 

In [38]:
do_unfollow_all_inactive_followings()

1  -  mamankia   is active.
2  -  Farzad6613297   is active.
